# Translation

In [1]:
import torch
import numpy as np

In [236]:
TRAIN_PERC = 0.6
VAL_PERC = 0.2
TEST_PERC = 0.2
VOCAB_SIZE = 10000
MODEL_DIM = 512
MAX_LENGTH = 1024

## Overview

The basic idea behind machine translation is to take a piece of text in one language and translate it to another language. In this example, we will look at translating English text into Welsh text.

In order to validate our approach, we need examples of English text that have been (correctly) translated to the corresponding Welsh.

## Corpus

### Download

In [ ]:
# https://techiaith.cymru/corpws/Moses/CofnodBachYCynulliad/CofnodBachYCynulliad.tar.gz
# https://techiaith.cymru/corpws/Moses/CofnodYCynulliad/CofnodYCynulliad.tar.gz

### Load

The corpus is generally too large to load entirely into memory, therefore we need to be able to load batches of translations dynamically.

In [216]:
class EnCyCorpus(torch.utils.data.IterableDataset):
    def __init__(self, en_corpus_file, cy_corpus_file):
        super(EnCyCorpus, self).__init__()
        self.en_corpus_file = en_corpus_file
        self.cy_corpus_file = cy_corpus_file

    def tidy(self, text):
        return text


    def __iter__(self):
        # Create an iterator
        en_itr = open(self.en_corpus_file)
        cy_itr = open(self.cy_corpus_file)

        # Map each element using the line_mapper
        mapped_en_itr = map(self.tidy, en_itr)
        mapped_cy_itr = map(self.tidy, cy_itr)

        # Zip both iterators
        zipped_itr = zip(mapped_en_itr, mapped_cy_itr)

        return zipped_itr

In [217]:
def load_corpus():
    # Load texts
    with open("translation/data/CofnodYCynulliad/CofnodYCynulliad.en", 'r', encoding='utf-8') as f:
        english_texts = f.read().splitlines()
    with open("translation/data/CofnodYCynulliad/CofnodYCynulliad.cy", 'r', encoding='utf-8') as f:
        welsh_texts = f.read().splitlines()
    texts = list(zip(english_texts, welsh_texts))

    # Split into train / val / test
    n_texts = len(texts)
    corpus = {}
    corpus['train'] = texts[:round(n_texts * TRAIN_PERC)]
    corpus['val'] = texts[round(n_texts * TRAIN_PERC):(round(n_texts * TRAIN_PERC) + round(n_texts * VAL_PERC))]
    corpus['test'] = texts[-round(n_texts * TEST_PERC):]

    # Sort by length to help with batching

    return corpus

In [218]:
corpus = load_corpus()

In [224]:
corpus

{'train': [('\ufeffthe presiding officer : i have a number of statements to make',
   "\ufeffy llywydd : mae gennyf nifer o ddatganiadau i'w gwneud"),
  ('for the question session that will follow , we have adopted a procedure that permits the question to be put in full',
   "ar gyfer y sesiwn holi a fydd yn dilyn , mae'r drefn a fabwysiadwyd yn caniatáu rhoi cwestiwn yn llawn"),
  ('this assists the spirit of openness , by conveying the contents of questions at the beginning of the discussion to other members , and to members of the public who are following our proceedings',
   "mae hyn yn ategu'r ysbryd o fod yn agored , gan gyfleu cynnwys y cwestiwn ar ddechrau'r drafodaeth i aelodau eraill , ac i aelodau'r cyhoedd sy'n dilyn ein gweithgareddau"),
  ('it has been brought to my attention that some members take advantage of this , and turn their questions into statements',
   "daeth i'm sylw fod rhai aelodau yn cymryd mantais o hyn , ac yn troi eu cwestiynau yn ddatganiadau"),
  ('fro

In [225]:
example_text = corpus['train'][1000:1002]
example_text

[('i hope that that will help create the right atmosphere when answering questions .',
  "gobeithiaf y bydd hynny o gymorth i greu'r awyrgylch iawn wrth ateb cwestiynau ."),
 ('extensive consultation was carried out with every relevant party , and the replies are being analysed at present .',
  'cafwyd ymgynghori helaeth gyda phob sefydliad perthnasol a dadansoddir yr atebion ar hyn o bryd .')]

## Model Inputs

The model needs the following as input:
1. The English text, converted into token indices
2. The output Welsh text, converted into token indices
3. The decoder input Welsh text, converted into token indices, which is a shifted version of the output.
4. An attention mask for the English text
5. An attention mask for the Welsh text

### Tokenizer

The first step in processing the text is to break it up into separate tokens, and assign an index to each token. These are typically sub-word level pieces of text. We will do this separately for both English and Welsh, since they clearly have different atomic tokens.

https://huggingface.co/course/chapter6/8?fw=pt#building-a-bpe-tokenizer-from-scratch

In [226]:
from tokenizers import Tokenizer
from tokenizers import models, pre_tokenizers, trainers, processors
from tokenizers import normalizers
from tokenizers import decoders
from transformers import PreTrainedTokenizerFast

In [227]:
def create_tokenizer(text):
    tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence(
        [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
    )
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
        [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
    )
    special_tokens = ["[BOS]", "[EOS]", "[PAD]", "[MASK]", "[UNK]"]
    tokenizer.model = models.WordPiece(unk_token="[UNK]")
    trainer = trainers.WordPieceTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
    tokenizer.train_from_iterator(text, trainer)
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[BOS] $A [EOS]",
        special_tokens=[
            ("[BOS]", tokenizer.token_to_id("[BOS]")),
            ("[EOS]", tokenizer.token_to_id("[EOS]")),
        ],
    )
    tokenizer.decoder = decoders.WordPiece(prefix="##")

    pretrained_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        bos_token="[BOS]",
        eos_token="[EOS]",
        pad_token="[PAD]",
        mask_token="[MASK]",
        unk_token="[UNK]",
    )
    return pretrained_tokenizer

In [228]:
english_tokenizer = create_tokenizer(
    text=[pair[0] for pair in corpus['train']]
)
welsh_tokenizer = create_tokenizer(
    text=[pair[1] for pair in corpus['train']]
)

In [229]:
example_tokenizer_output = english_tokenizer(
    text=[ex[0] for ex in example_text],
    return_token_type_ids=False, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [230]:
example_tokenizer_output

{'input_ids': tensor([[   0,   45,  741,  131,  131,  167,  813, 2058,  112,  671, 8549,  435,
         5596, 1245,   18,    1,    2,    2,    2,    2,    2,    2],
        [   0, 4402, 1068,  250, 2665,  398,  202,  697, 2062,  768,   16,  136,
          112, 7402,  170,  583, 9528,   83,  246, 1028,   18,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [231]:
english_tokenizer.batch_decode(example_tokenizer_output['input_ids'], clean_up_tokenization_spaces=False)

['[BOS] i hope that that will help create the right atmosphere when answering questions. [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[BOS] extensive consultation was carried out with every relevant party, and the replies are being analysed at present. [EOS]']

In [232]:
english_tokenizer.convert_ids_to_tokens(example_tokenizer_output['input_ids'][1])

['[BOS]',
 'extensive',
 'consultation',
 'was',
 'carried',
 'out',
 'with',
 'every',
 'relevant',
 'party',
 ',',
 'and',
 'the',
 'replies',
 'are',
 'being',
 'analyse',
 '##d',
 'at',
 'present',
 '.',
 '[EOS]']

### Collator

In general, we need to deal with batches of input, and for training the model it is more efficient. The input to the model is:
1. All the English tokens
2. All the Welsh tokens
3. The source attention mask to tell it which source tokens to pay attention to
4. The target attention mask to tell it which target tokens to pay attention to
5. The decoder input ids, which are right-shifted target tokens


In [237]:
def collate_batch(texts):
    src_batch = english_tokenizer(
        text=[text[0] for text in texts],
        return_token_type_ids=False,
        padding=True,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True,
        max_length=MAX_LENGTH
    )
    src_batch = {'src_' + str(k): v for k,v in src_batch.items()}
    tgt_batch = welsh_tokenizer(
        text=[text[1] for text in texts],
        return_token_type_ids=False,
        padding=True,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True,
        max_length=MAX_LENGTH
    )
    tgt_batch = {'tgt_' + str(k): v for k,v in tgt_batch.items()}
    tgt_batch['tgt_output_ids'] = tgt_batch['tgt_input_ids'][:, 1:]
    tgt_batch['tgt_input_ids'] = tgt_batch['tgt_input_ids'][:, :-1]
    tgt_batch['tgt_attention_mask'] = tgt_batch['tgt_attention_mask'][:, :-1]

    # Extend shortest input
    src_shape = src_batch['src_input_ids'].shape
    tgt_shape = tgt_batch['tgt_input_ids'].shape
    if src_shape[1] < tgt_shape[1]:
        diff = tgt_shape[1] - src_shape[1]
        src_batch['src_input_ids'] = torch.cat((src_batch['src_input_ids'], torch.full([src_shape[0],diff], 2)), dim=1)
        src_batch['src_attention_mask'] = torch.cat((src_batch['src_attention_mask'], torch.full([src_shape[0],diff], 0)), dim=1)
    elif tgt_shape[1] < src_shape[1]:
        diff = src_shape[1] - tgt_shape[1]
        tgt_batch['tgt_input_ids'] = torch.cat((tgt_batch['tgt_input_ids'], torch.full([tgt_shape[0],diff], 2)), dim=1)
        tgt_batch['tgt_attention_mask'] = torch.cat((tgt_batch['tgt_attention_mask'], torch.full([tgt_shape[0],diff], 0)), dim=1)
        tgt_batch['tgt_output_ids'] = torch.cat((tgt_batch['tgt_output_ids'], torch.full([tgt_shape[0],diff], 2)), dim=1)

    # Combine
    batch = {**src_batch, **tgt_batch}
    return batch

In [238]:
example_batch = collate_batch(example_text)

In [239]:
example_batch

{'src_input_ids': tensor([[   0,   45,  741,  131,  131,  167,  813, 2058,  112,  671, 8549,  435,
          5596, 1245,   18,    1,    2,    2,    2,    2,    2,    2],
         [   0, 4402, 1068,  250, 2665,  398,  202,  697, 2062,  768,   16,  136,
           112, 7402,  170,  583, 9528,   83,  246, 1028,   18,    1]]),
 'src_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'tgt_input_ids': tensor([[   0, 1070,   63,  195,  196,   53, 1577,   47, 1842,   11,   56, 9015,
           583,  326,  682, 1735,   18,    1,    2,    2,    2,    2],
         [   0, 2704, 1213, 2214,  432, 3405, 1984, 2936,   39, 7446,   80,  153,
          2502,  132,  161,   53,  558,   18,    2,    2,    2,    2]]),
 'tgt_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 

This is the input that our model will receive, together with the outcome it will be trained against.

## Model

In [240]:
import torch.nn as nn

The model we will build is a canonical encoder-decoder model. On the encoder side, a representation of the input is created, and on the decoder side this representation is used to generate a sequence of tokens in an autoregressive way.

### Encoder

The encoder is a stack of N transformer layers, each of which is composed of a self-attention layer with a dense layer, including a residual connection. Let's start by defining the general Encoder:

#### Embedding

An embedding takes a sequence of token ids and converts it into a sequence of n-dimensional vector representations of each token. Both the encoder and decoder have separate embeddings, since they deal with separate languages.

We will use the pytorch Embedding [https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html].

In [241]:
import math

class Embeddings(nn.Module):
    def __init__(self, d_model=512, n_vocab=20000):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(n_vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

Let's see how this looks for our example batch:

In [242]:
example_embedding_output = Embeddings()(example_batch
['src_input_ids'])
example_embedding_output

tensor([[[-25.8897, -39.9070, -14.5269,  ..., -26.1514,  -6.6897,  -2.0937],
         [ 12.9494, -15.0668,   2.8416,  ...,  -2.0395,  -0.1918, -26.9800],
         [ -3.0216, -13.3112,  -7.1148,  ...,  14.4628,  10.6719,  -2.4404],
         ...,
         [ 33.0440,  -3.4465, -11.5646,  ...,  19.6917,  18.8510,  17.0128],
         [ 33.0440,  -3.4465, -11.5646,  ...,  19.6917,  18.8510,  17.0128],
         [ 33.0440,  -3.4465, -11.5646,  ...,  19.6917,  18.8510,  17.0128]],

        [[-25.8897, -39.9070, -14.5269,  ..., -26.1514,  -6.6897,  -2.0937],
         [ 19.2847,  13.0735, -11.0542,  ...,   3.8324,  -6.8153,  -7.6713],
         [-33.6921, -20.2254,  35.9021,  ...,   7.8319,  13.2310,  24.9708],
         ...,
         [ 44.5540,  -6.1444,  23.3959,  ...,  24.8200,   4.2109,  23.9702],
         [  5.9880,  -0.1729,  -4.2262,  ...,   0.6301, -14.1546,   8.1401],
         [-12.7087,  -6.5566, -18.9570,  ..., -18.0415,  -3.4669,  22.9071]]],
       grad_fn=<MulBackward0>)

In [97]:
example_embedding_output.size()

torch.Size([2, 22, 512])

We have converted each token in the batch into a vector of dimension MODEL_DIM. The first rank of the tensor represents the batch no, the second is the token no and the third is the vector coefficient of the embedding.

#### Positional Encoding

In order to tell the model which position each token is in, the token embeddings have to be augmented with position information. We will use a common method that 

In [243]:
from torch.autograd import Variable

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model=512, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)],
                         requires_grad=False)
        return self.dropout(x)

In [244]:
example_pos_enc_output = PositionalEncoding()(example_embedding_output)
example_pos_enc_output

tensor([[[-28.7663, -43.2300, -16.1410,  ..., -27.9460,  -7.4330,  -1.2152],
         [ 15.3232, -16.1405,   4.0705,  ...,  -1.1549,  -0.2130, -28.8667],
         [ -2.3471, -15.2526,  -6.8648,  ...,   0.0000,  11.8579,  -1.6004],
         ...,
         [ 36.8821,  -2.7309, -13.4025,  ...,  22.9908,  20.9478,   0.0000],
         [ 37.7300,  -3.3760, -12.3726,  ...,  22.9908,  20.9479,  20.0142],
         [ 37.6452,  -0.0000, -11.7530,  ...,  22.9908,  20.9480,  20.0142]],

        [[-28.7663, -43.2300, -16.1410,  ..., -27.9460,  -7.4330,  -1.2152],
         [ 22.3624,  15.1265, -11.3692,  ...,   5.3694,  -7.5725,  -7.4125],
         [-36.4253, -22.9350,  40.9317,  ...,   9.8132,  14.7013,  28.8565],
         ...,
         [ 49.6710,  -5.7286,  25.4425,  ...,   0.0000,   4.6810,   0.0000],
         [  7.6677,   0.2613,  -4.2188,  ...,   1.8112, -15.7251,  10.1557],
         [-13.1912,  -7.8937, -19.9668,  ..., -18.9350,  -3.8497,  26.5634]]],
       grad_fn=<MulBackward0>)

In [328]:
example_pos_enc_output.size()

torch.Size([2, 22, 512])

The size of the tensor is the same as before, since all that has been added is some positional information to each dimension of each token.

#### Encoder Layer

So far we have transformed each input tokens into a vector representing that token, via an embedding, and added positional information, via the positional encoder. Now, the role of the encoder is to represent the input in such a way that the decoder can make best use of it. In order to do this, each layer of the encoder "shares" semantic information between tokens.

#### Multi-Headed Attention

Multi-headed attention is the mechanism by which this sharing takes place. It allows each token to learn how to "pay attention" to other tokens, and gradually assimilate their semantic meaning into their representations. Since the model has to deal with sequences of varying length, the attention mechanism has to learn how to first assess the relationship between tokens, and then how to share their representations i.e. it cannot say "mix token 1 with token 3".

In [101]:
import copy

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [102]:
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask.unsqueeze(2) == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [103]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

To see the output of the multi-headed attention we need to feed it some input. We can feed it separate values for Q, K, and V. For the encoder, these will all be the representations of the tokens, hence the term "self-attention". We also input the attention mask, since we need to tell the attention layer to ignore any tokens that represent padding, coming from our batch.

In [105]:
example_tokenizer_output

{'input_ids': tensor([[   0,   45,  741,  131,  131,  167,  813, 2058,  112,  671, 8550,  435,
         5596, 1245,   18,    1,    2,    2,    2,    2,    2,    2],
        [   0, 4402, 1068,  250, 2665,  398,  202,  697, 2062,  768,   16,  136,
          112, 7402,  170,  583, 9526,   91,  246, 1028,   18,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [107]:
example_attn_output = MultiHeadedAttention(h=8, d_model=MODEL_DIM, dropout=0.1)(
    example_pos_enc_output,
    example_pos_enc_output,
    example_pos_enc_output,
    example_batch['src_attention_mask']
)
example_attn_output

tensor([[[  0.1920,  -8.0616,  12.1870,  ...,   4.6376,  -6.3434,   3.2600],
         [ 16.3910,  -4.1503,  17.4403,  ...,  -3.6862,  16.8644,  -7.1311],
         [  9.2246,  -4.8245,  -5.3588,  ...,   3.9072,   2.9700,   2.0772],
         ...,
         [  0.5158,  -0.8326,   3.0285,  ...,   5.6848,  13.2035, -10.7891],
         [-10.9216,   6.9987,  -1.1041,  ...,   2.6961,   3.1806,  -5.1459],
         [  1.2143,   9.1400, -12.1723,  ...,  13.5675,  11.1429, -12.4582]],

        [[ -1.4643,   2.9577,   4.0780,  ...,  -5.3677,  -0.9760,  -0.2644],
         [ -5.8037,  -2.5812,  13.9444,  ...,   1.3678,  -7.1553,  -4.0912],
         [ -1.2225,  -8.0255,  -7.0852,  ...,   1.7904,  16.1246,   0.4308],
         ...,
         [  6.9562,   2.9363,   2.8264,  ...,  -1.5850, -15.3895,  -8.1148],
         [ 11.7618,  12.5986,   2.7443,  ...,  -7.7506, -12.2355,  17.5846],
         [  7.8088,   0.5737,  -4.4561,  ...,   0.6292,   2.3043,  10.7107]]],
       grad_fn=<ViewBackward0>)

In [108]:
example_attn_output.size()

torch.Size([2, 22, 512])

#### Layer Normalisation

In [109]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

#### Sub-layer Connection

In [110]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [111]:
example_attn_sublayer_output = SublayerConnection(size=MODEL_DIM, dropout=0.1)(
    example_pos_enc_output,
    lambda x: MultiHeadedAttention(h=8, d_model=MODEL_DIM, dropout=0.1)(x, x, x, example_batch['src_attention_mask'])
)
example_attn_sublayer_output

tensor([[[ 1.8614e-01,  2.8400e+01, -1.6949e-01,  ..., -9.2039e+00,
          -4.8522e+01, -5.2071e+01],
         [-2.1234e+01, -1.1041e+01,  5.3760e-01,  ..., -1.0642e+01,
          -3.7987e+01, -5.1724e+01],
         [ 3.7165e+01, -2.9675e+00, -4.3879e+00,  ..., -7.9804e+00,
          -3.5800e+01, -3.8595e+01],
         ...,
         [ 2.0313e+01,  1.9306e-01, -3.7872e+00,  ...,  9.4374e+00,
          -1.0144e+01,  4.0170e+01],
         [ 2.1145e+01, -1.7933e+01, -2.7651e+00,  ...,  9.4319e+00,
          -1.0148e+01,  4.0140e+01],
         [ 2.1054e+01, -1.9154e+01, -2.1684e+00,  ...,  9.4140e+00,
          -1.0086e+01,  4.0146e+01]],

        [[-3.8584e+01,  2.8407e+01, -4.5539e+01,  ..., -9.1606e+00,
          -4.8317e+01, -1.0985e-01],
         [ 5.3257e+01,  1.0627e+00,  2.0269e+01,  ..., -3.7417e+01,
          -2.8004e+01, -1.0899e+01],
         [-2.4751e+01,  1.5936e+01, -9.3505e+00,  ..., -3.4933e-02,
           1.5455e+01,  2.8899e+01],
         ...,
         [-3.4146e+01, -2

In [112]:
example_attn_sublayer_output.shape

torch.Size([2, 22, 512])

#### Position-Wise Feed-forward Layer

Each token vector is then fed through an identical densely connected layer.

In [113]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [114]:
example_ffn_output = PositionwiseFeedForward(d_model=MODEL_DIM, d_ff=2048, dropout=0.1)(
    example_attn_sublayer_output
)
example_ffn_output

tensor([[[ 3.1836e+00, -9.2368e+00, -4.3725e+00,  ...,  5.4393e+00,
          -5.9997e-01, -2.5736e+00],
         [-1.3680e+01, -6.1491e+00, -5.7365e-01,  ...,  8.4583e+00,
          -2.4544e+00, -1.9194e+00],
         [ 9.5345e+00,  2.9896e-02,  5.6210e+00,  ..., -2.4381e+00,
           2.2146e+00,  9.3947e-01],
         ...,
         [ 7.2934e+00, -3.4986e+00, -7.3746e-01,  ...,  1.2000e+01,
          -1.2271e+00,  8.5686e+00],
         [ 6.2151e+00, -1.9886e-01, -1.5534e+00,  ...,  5.8545e+00,
          -6.2957e+00,  2.4263e+00],
         [ 9.1377e+00, -3.8672e+00, -2.5255e+00,  ...,  6.0578e+00,
          -1.5369e+00,  4.7688e-01]],

        [[ 6.6993e+00, -9.3848e+00, -5.5839e+00,  ...,  5.6225e+00,
          -4.2399e-01,  1.0231e+00],
         [-8.7006e+00, -6.3309e+00,  4.5514e-04,  ...,  3.8513e+00,
           5.5180e+00, -8.3046e+00],
         [ 5.8206e+00,  2.4432e-01, -1.8336e-01,  ..., -4.1959e+00,
           6.0570e+00,  1.3539e+00],
         ...,
         [-3.9602e+00, -4

In [115]:
example_ffn_output.shape

torch.Size([2, 22, 512])

This is also wrapped in a residual connection:

In [116]:
example_sublayer_ffn_output = SublayerConnection(size=MODEL_DIM, dropout=0.1)(
    example_attn_sublayer_output,
    lambda x: PositionwiseFeedForward(d_model=MODEL_DIM, d_ff=2048, dropout=0.1)(x)
)
example_sublayer_ffn_output

tensor([[[ -0.1032,  28.4999,   0.0797,  ...,  -9.2788, -48.4657, -52.7605],
         [-21.2607, -10.9605,   0.9027,  ..., -10.7737, -38.0838, -52.2825],
         [ 36.9496,  -3.2906,  -4.1375,  ...,  -7.4094, -35.9227, -38.6707],
         ...,
         [ 19.8028,   0.5374,  -3.5848,  ...,   9.6251, -10.3398,  40.1705],
         [ 20.6546, -17.6744,  -2.5636,  ...,   9.5368, -10.2153,  39.4363],
         [ 21.0540, -19.0612,  -1.9367,  ...,   9.5919, -10.2420,  39.5824]],

        [[-38.9004,  28.5042, -45.5385,  ...,  -9.3385, -48.1295,  -0.7858],
         [ 53.2567,   1.1497,  20.3706,  ..., -37.2545, -28.0045, -11.0479],
         [-24.9864,  15.9362,  -9.3149,  ...,   0.3484,  15.3407,  28.4971],
         ...,
         [-34.1461, -20.8889,  29.4540,  ...,  11.4437,  -9.4731,   0.0800],
         [ -0.1019,   0.3047,  16.0980,  ..., -38.8291, -24.0562,   3.8281],
         [ -0.4796, -15.5397,  -0.2046,  ...,  54.3686,  30.7018,  -8.8193]]],
       grad_fn=<AddBackward0>)

In [117]:
example_sublayer_ffn_output.shape

torch.Size([2, 22, 512])

#### Encoder Layer

And that's it! We've reached the end of our first encoder layer. Putting it all together each encoder layer is:

In [166]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

#### Encoder

And the full encoder is:

In [167]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [168]:
c = copy.deepcopy
attn = MultiHeadedAttention(h=2, d_model=MODEL_DIM)
ff = PositionwiseFeedForward(d_model=MODEL_DIM, d_ff=2048, dropout=0.1)
dropout = 0.1
example_encoder_output = Encoder(
    layer=EncoderLayer(
        size=MODEL_DIM,
        self_attn=c(attn),
        feed_forward=c(ff),
        dropout=0.1),
    N=2)(example_pos_enc_output, example_batch['src_attention_mask'])
example_encoder_output

tensor([[[ 1.4310e-02,  1.1522e+00,  2.8671e-02,  ..., -4.1057e-01,
          -1.9842e+00, -2.1611e+00],
         [-9.1315e-01, -4.9973e-01,  5.9216e-04,  ..., -5.2752e-01,
          -1.5928e+00, -2.1544e+00],
         [ 1.5835e+00, -8.1105e-02, -1.8206e-02,  ..., -2.4071e-01,
          -1.3675e+00, -1.5165e+00],
         ...,
         [ 8.2074e-01, -3.5501e-02, -2.0296e-01,  ...,  3.3824e-01,
          -4.5953e-01,  1.6582e+00],
         [ 8.8238e-01, -7.9344e-01, -1.2690e-01,  ...,  3.6146e-01,
          -4.5266e-01,  1.7036e+00],
         [ 8.2586e-01, -8.6521e-01, -1.2931e-01,  ...,  3.3479e-01,
          -4.5858e-01,  1.6509e+00]],

        [[-1.5873e+00,  1.1978e+00, -1.8639e+00,  ..., -4.5616e-01,
          -2.0127e+00, -4.8626e-02],
         [ 2.2002e+00, -1.3161e-01,  8.7085e-01,  ..., -1.6729e+00,
          -1.2538e+00, -5.0743e-01],
         [-1.0651e+00,  6.1448e-01, -3.9876e-01,  ..., -5.0641e-02,
           6.5514e-01,  1.2407e+00],
         ...,
         [-1.4755e+00, -9

In [169]:
example_encoder_output.shape

torch.Size([2, 22, 512])

### Decoder

The input to the decoder is:
1. The output of the encoder
2. The previous tokens in the sequence up to that point
3. The source mask showing which source tokens the decoder can pay attention to
4. The target mask showing which target tokens the decoder can pay attention to

In [170]:
example_batch

{'src_input_ids': tensor([[   0,   45,  741,  131,  131,  167,  813, 2058,  112,  671, 8550,  435,
          5596, 1245,   18,    1,    2,    2,    2,    2,    2,    2],
         [   0, 4402, 1068,  250, 2665,  398,  202,  697, 2062,  768,   16,  136,
           112, 7402,  170,  583, 9526,   91,  246, 1028,   18,    1]]),
 'src_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'tgt_input_ids': tensor([[   0, 1070,   63,  195,  196,   53, 1577,   47, 1842,   11,   56, 9015,
           583,  326,  682, 1735,   18,    1,    2,    2,    2,    2],
         [   0, 2704, 1213, 2214,  432, 3405, 1984, 2936,   39, 7446,   83,  153,
          2502,  132,  161,   53,  558,   18,    2,    2,    2,    2]]),
 'tgt_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 

In [171]:
example_tgt_embedding_output = Embeddings(512, VOCAB_SIZE)(example_batch['tgt_input_ids'])
example_tgt_pos_enc_output = PositionalEncoding(512,0.1)(example_tgt_embedding_output)
example_tgt_pos_enc_output

tensor([[[  5.8286,  -0.0000,   9.6820,  ...,  -9.0398, -13.3486,   8.4563],
         [ 24.2495, -19.6370,  -2.5031,  ..., -32.5745, -12.0261, -14.8670],
         [-19.6888,  13.1659,   2.4430,  ...,  15.8603, -32.6508,  28.8665],
         ...,
         [ -5.5493,  26.5495,  -5.5052,  ..., -14.4604,   3.0550,  14.3575],
         [ -4.7014,  25.9044,  -4.4752,  ...,  -0.0000,   3.0551,  14.3575],
         [ -4.7862,  24.8424,  -3.8557,  ..., -14.4604,   3.0553,  14.3575]],

        [[  5.8286,  -1.4550,   9.6820,  ...,  -9.0398,  -0.0000,   8.4563],
         [ -0.0000,   3.6290, -18.0780,  ...,  -3.6336,  22.4507,  10.6727],
         [ -0.0000, -16.1577,   1.7718,  ...,   0.0000, -36.5178, -72.5394],
         ...,
         [ -5.5493,  26.5495,  -5.5052,  ..., -14.4604,   3.0550,  14.3575],
         [ -4.7014,  25.9044,  -4.4752,  ..., -14.4604,   3.0551,  14.3575],
         [ -4.7862,  24.8424,  -3.8557,  ..., -14.4604,   3.0553,  14.3575]]],
       grad_fn=<MulBackward0>)

In [172]:
example_tgt_pos_enc_output.shape

torch.Size([2, 22, 512])

#### Decoder Layer

In [175]:
MultiHeadedAttention(h=8, d_model=512)(
    example_tgt_pos_enc_output,
    example_tgt_pos_enc_output,
    example_tgt_pos_enc_output,
    example_batch['tgt_attention_mask']
)

tensor([[[ 3.4241e+00,  1.5115e+01, -7.5266e+00,  ..., -1.1657e+01,
          -1.0247e+01, -9.8763e-03],
         [-1.2680e+01,  8.4389e+00, -3.0016e+00,  ..., -1.1956e+01,
          -2.4363e+00,  5.6326e+00],
         [ 1.7149e-01,  9.6464e+00, -1.8841e+00,  ...,  6.1402e+00,
           4.8300e+00,  7.2312e+00],
         ...,
         [-2.1939e+00, -1.1523e+01, -2.7049e+00,  ...,  1.7027e+00,
           6.4457e+00, -8.6503e+00],
         [-1.0129e+01, -1.4715e+01, -6.6962e+00,  ...,  5.7623e+00,
           1.5422e+01, -1.1384e+01],
         [-6.6961e+00, -2.2064e+00,  7.2900e-02,  ..., -8.6733e+00,
           1.0138e+01, -4.9338e+00]],

        [[-4.9196e-01,  1.0320e+01, -6.2101e+00,  ..., -4.4297e+00,
          -6.3575e+00, -4.8989e+00],
         [-1.0765e+01, -2.1929e-01, -2.9287e+00,  ..., -8.0981e+00,
          -2.1598e+00,  4.1278e-01],
         [-1.2973e+00, -4.7371e+00,  2.0007e+01,  ..., -7.0164e+00,
           1.6256e+01,  1.0959e+01],
         ...,
         [ 3.3669e+00, -2

In [176]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [178]:
c = copy.deepcopy
attn = MultiHeadedAttention(h=8, d_model=512)
ff = PositionwiseFeedForward(d_model=512, d_ff=2048, dropout=0.1)
position = PositionalEncoding(d_model=512, dropout=0.1)
DecoderLayer(512, c(attn), c(attn), c(ff), dropout=0.1)(
    x=example_tgt_pos_enc_output,
    memory=example_encoder_output,
    src_mask=example_batch['src_attention_mask'],
    tgt_mask=example_batch['tgt_attention_mask']
)

tensor([[[ 6.0497e+00, -1.6359e-02,  9.2981e+00,  ..., -8.9984e+00,
          -1.3124e+01,  8.7872e+00],
         [ 2.4777e+01, -2.0105e+01, -2.6817e+00,  ..., -3.2752e+01,
          -1.1998e+01, -1.4635e+01],
         [-1.9287e+01,  1.2844e+01,  2.7165e+00,  ...,  1.6053e+01,
          -3.2761e+01,  2.9111e+01],
         ...,
         [-5.5654e+00,  2.6423e+01, -5.4091e+00,  ..., -1.4361e+01,
           2.7273e+00,  1.4192e+01],
         [-4.6789e+00,  2.5582e+01, -4.2727e+00,  ...,  2.1494e-01,
           2.8010e+00,  1.4332e+01],
         [-4.7660e+00,  2.4532e+01, -3.5117e+00,  ..., -1.4462e+01,
           2.4417e+00,  1.4431e+01]],

        [[ 6.7228e+00, -1.3901e+00,  9.5352e+00,  ..., -8.8095e+00,
          -5.9263e-02,  8.7638e+00],
         [-3.6100e-01,  3.1946e+00, -1.7741e+01,  ..., -3.3016e+00,
           2.2021e+01,  1.0593e+01],
         [ 8.5785e-01, -1.5989e+01,  2.0714e+00,  ..., -1.3277e-01,
          -3.6230e+01, -7.2237e+01],
         ...,
         [-5.5369e+00,  2

#### Decoder

In [179]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [180]:
c = copy.deepcopy
attn = MultiHeadedAttention(h=8, d_model=512)
ff = PositionwiseFeedForward(d_model=512, d_ff=2048, dropout=0.1)
position = PositionalEncoding(d_model=512, dropout=0.1)
example_decoder_output = Decoder(
    layer=DecoderLayer(512, c(attn), c(attn), c(ff), dropout=0.1),
    N=2
)(x=example_tgt_pos_enc_output,
    memory=example_encoder_output,
    src_mask=example_batch['src_attention_mask'],
    tgt_mask=example_batch['tgt_attention_mask'])

In [181]:
example_decoder_output

tensor([[[ 0.2352,  0.0194,  0.3702,  ..., -0.3726, -0.5356,  0.3208],
         [ 1.0193, -0.7917, -0.0595,  ..., -1.3356, -0.4542, -0.6223],
         [-0.9125,  0.5169,  0.0792,  ...,  0.6712, -1.4291,  1.2254],
         ...,
         [-0.2761,  1.0254, -0.3166,  ..., -0.6078,  0.0334,  0.5175],
         [-0.2166,  1.0071, -0.2656,  ..., -0.0140,  0.0710,  0.5321],
         [-0.2678,  0.9575, -0.2353,  ..., -0.6234,  0.0465,  0.5276]],

        [[ 0.2201, -0.0543,  0.3694,  ..., -0.4048,  0.0091,  0.3324],
         [-0.0220,  0.1297, -0.6954,  ..., -0.1551,  0.9529,  0.4282],
         [-0.0164, -0.7912,  0.0068,  ..., -0.1014, -1.6373, -3.1989],
         ...,
         [-0.2698,  1.0694, -0.2796,  ..., -0.6404,  0.0931,  0.5407],
         [-0.2630,  1.0132, -0.2573,  ..., -0.6471,  0.0601,  0.5237],
         [-0.2471,  1.0049, -0.2273,  ..., -0.6291,  0.0848,  0.5633]]],
       grad_fn=<AddBackward0>)

In [182]:
example_decoder_output.shape

torch.Size([2, 22, 512])

#### Generator

The final step is to take the output of the decoder and predict the token.

In [141]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [183]:
example_generator_output = Generator(d_model=512, vocab=VOCAB_SIZE)(example_decoder_output)

In [184]:
example_generator_output.shape

torch.Size([2, 22, 10000])

Note that the generator takes in the decoder output for each step, and maps the decoder vector representation for each token to a probability for *all* tokens in the sequence, not just the last one. For example, the first sequence, first token output log probabilities are:

In [185]:
example_generator_output[0, 0, :]

tensor([ -9.7038,  -9.0570,  -9.3031,  ..., -10.0155,  -8.9369,  -8.6091],
       grad_fn=<SliceBackward0>)

The most likely token is therefore the one with the highest probability:

In [186]:
vocab_ind = example_generator_output[0, 0, :].argmax()
vocab_ind

tensor(6807)

In [187]:
welsh_tokenizer.decode(vocab_ind)

'ymad'

### Encoder-Decoder

Putting the full model together gives:

In [188]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [189]:
def make_model(src_vocab, tgt_vocab, N=6,
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn),
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [376]:
example_model = make_model(VOCAB_SIZE, VOCAB_SIZE, 2)

/tmp/ipykernel_1307/320853634.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [377]:
example_model_output = make_model(VOCAB_SIZE, VOCAB_SIZE, 2)(
    example_batch['src_input_ids'], example_batch['tgt_input_ids'], example_batch['src_attention_mask'], example_batch['tgt_attention_mask'])
example_model_output

/tmp/ipykernel_1307/320853634.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [378]:
example_model_output.shape

torch.Size([2, 22, 512])

The forward pass of the model doesn't use the Generator?

## Training

In [379]:
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from torch.optim.lr_scheduler import LambdaLR
#import GPUtil
import time

In order to train the model we need to decide on a few training parameters.

### Loss Function

In [380]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

In [381]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

### Optimizer

In [382]:
example_optimizer = torch.optim.Adam(
        example_model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9
)

In [197]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

In [198]:
example_lr_scheduler = LambdaLR(
    optimizer=example_optimizer,
    lr_lambda=lambda step: rate(
        step, 512, factor=1, warmup=100
    ),
)

In [199]:
class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None

In [200]:
class DummyScheduler:
    def step(self):
        None

### Epoch

In [368]:
train_dataloader = torch.utils.data.DataLoader(corpus['train'], batch_size=64, collate_fn=collate_batch)
val_dataloader = torch.utils.data.DataLoader(corpus['val'], batch_size=64, collate_fn=collate_batch)
test_dataloader = torch.utils.data.DataLoader(corpus['test'], batch_size=64, collate_fn=collate_batch)

In [369]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed

In [370]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch['src_input_ids'],
            batch['tgt_input_ids'],
            batch['src_attention_mask'],
            batch['tgt_attention_mask']
        )
        ntokens = (batch['tgt_output_ids'] != 2).data.sum()
        loss, loss_node = loss_compute(out, batch['tgt_output_ids'], ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch['src_input_ids'].shape[0]
            train_state.tokens += ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()

        total_loss += loss
        total_tokens += ntokens
        tokens += ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
    return total_loss / total_tokens, train_state

In [367]:
run_epoch(
    data_iter=train_dataloader,
    model=example_model,
    loss_compute=SimpleLossCompute(example_model.generator, LabelSmoothing(
        size=VOCAB_SIZE, padding_idx=2, smoothing=0.1
    )),
    optimizer=example_optimizer,
    scheduler=example_lr_scheduler,
    mode="train"
)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

### Training Loop

In [ ]:
torch.set_default_device("cuda:0" if torch.cuda.is_available() else "cpu")

In [363]:
def train_worker(
    gpu,
    config
):
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.device(gpu)

    pad_idx = 2
    d_model = 512
    model = make_model(VOCAB_SIZE, VOCAB_SIZE, N=6)
    model.cuda(gpu)
    module = model
    is_main_process = True

    criterion = LabelSmoothing(
        size=VOCAB_SIZE, padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)

    # train_dataloader, valid_dataloader = create_dataloaders(
    #     gpu,
    #     vocab_src,
    #     vocab_tgt,
    #     spacy_de,
    #     spacy_en,
    #     batch_size=config["batch_size"] // ngpus_per_node,
    #     max_padding=config["max_padding"],
    #     is_distributed=is_distributed,
    # )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState()

    for epoch in range(config["num_epochs"]):
        model.train()
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            train_dataloader,
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        #GPUtil.showUtilization()
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            val_dataloader,
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        print(sloss)
        torch.cuda.empty_cache()

    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path)

In [364]:
config = {
    "batch_size": 32,
    "distributed": False,
    "num_epochs": 1,
    "accum_iter": 10,
    "base_lr": 1.0,
    "max_padding": 72,
    "warmup": 3000,
    "file_prefix": "english_welsh_model_",
    }

In [366]:
train_worker(-1, config)

Train worker process using GPU: -1 for training


/tmp/ipykernel_1307/320853634.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


RuntimeError: Device index must not be negative

# Predicting

With the model built and trained, we can use it to translate sentences in our test corpus from English to Welsh

In [310]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [314]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    tgt = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory = memory, src_mask = src_mask,
                           tgt = Variable(tgt),
                           tgt_mask = Variable(subsequent_mask(tgt.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        tgt = torch.cat([tgt,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return tgt

In [315]:
example_tokenizer_output

{'input_ids': tensor([[   0,   45,  741,  131,  131,  167,  813, 2058,  112,  671, 8549,  435,
         5596, 1245,   18,    1,    2,    2,    2,    2,    2,    2],
        [   0, 4402, 1068,  250, 2665,  398,  202,  697, 2062,  768,   16,  136,
          112, 7402,  170,  583, 9528,   83,  246, 1028,   18,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [330]:
model = example_model

src = example_batch['src_input_ids'][0:1, ]
src_mask = example_batch['src_attention_mask'][0:1, ]
max_len=20
start_symbol=0

pred_tokens = greedy_decode(model, src, src_mask, max_len, start_symbol)
pred_text = welsh_tokenizer.batch_decode(pred_tokens)
actual_text = welsh_tokenizer.batch_decode(example_batch['tgt_input_ids'][0:1, ])
print(english_tokenizer.batch_decode(example_batch['src_input_ids'][0:1, ]))
print(actual_text)
print(pred_text)


['[BOS] i hope that that will help create the right atmosphere when answering questions. [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']
["[BOS] gobeithiaf y bydd hynny o gymorth i greu'r awyrgylch iawn wrth ateb cwestiynau. [EOS] [PAD] [PAD] [PAD] [PAD]"]
['[BOS] paragraff gofynnodd oaq20 ddirbertbert rheoli ebrillymod rogers rogers rogers rogers rogers rogers rogers rogers rogers rogers']


In [383]:
test_example = collate_batch(corpus['test'][1000:1001])
pred_tokens = greedy_decode(example_model, test_example['src_input_ids'], test_example['src_attention_mask'], max_len=20, start_symbol=0)
pred_text = welsh_tokenizer.batch_decode(pred_tokens)
actual_text = welsh_tokenizer.batch_decode(test_example['tgt_input_ids'][0:1, ])
print(english_tokenizer.batch_decode(test_example['src_input_ids'][0:1, ]))
print(actual_text)
print(pred_text)

['[BOS] when we read that, we thought that it meant that the formula would be reviewed upwards, not downwards as has been the case in the last two years. [EOS] [PAD] [PAD] [PAD] [PAD]']
['[BOS] pan ddarllenasom hynny, yr oeddem yn meddwl ei fod yn golygu y cai ’ r fformiwla ei adolygu tuag i fyny, nid tuag i lawr fel sydd wedi digwydd yn y ddwy flynedd diwethaf.']
['[BOS]efyd hwyr fis funud gynhyrchu camarwain pethau funud dren cambri disgybl ymgysylltu edifar hoffi ieith cambri gohirio disgyblellach']
